In [1]:
from IPython.display import display,HTML,clear_output
import json

# Read explanation report

In [2]:
explanation_report_filepath = "/Users/salvatore/PycharmProjects/T-EBAnO-Express/outputs/20201117_bert_model_imdb_reviews_exp_0/local_explanations_experiments/20201204_105147/local_explanations/local_explanation_report_0.json"


In [3]:
with open(explanation_report_filepath) as json_file:
    explanation_report = json.load(json_file)

# Parse explanation report

In [4]:
input_info = explanation_report["input_info"]
input_positions_tokens = input_info["positions_tokens"]
raw_text = input_info["raw_text"]
cleaned_text = input_info["cleaned_text"]
preprocessed_text = input_info["preprocessed_text"]

In [5]:
pos_local_explanations = []  # pos features without combinations
sen_local_explanations = []  # sen features without combinations
mlwe_local_explanations = []  # mlwe features without combinations
pos_local_explanations_comb = []  # All pos features (also combinations)
sen_local_explanations_comb = []  # All sen features (also combinations)
mlwe_local_explanations_comb = []  # All mlwe features (also combinations)

for le in explanation_report["local_explanations"]:
    if le["feature_type"] == "POS" and le["combination"] == 1:
        pos_local_explanations.append(le)
    if le["feature_type"] == "SEN" and le["combination"] == 1:
        sen_local_explanations.append(le)
    if le["feature_type"] == "MLWE" and le["combination"] == 1:
        mlwe_local_explanations.append(le)
    if le["feature_type"] == "POS":
        pos_local_explanations_comb.append(le)
    if le["feature_type"] == "SEN":  
        sen_local_explanations_comb.append(le)
    if le["feature_type"] == "MLWE":
        mlwe_local_explanations_comb.append(le)
        

In [6]:
pos_positions_tokens_score = {} # Dictionary for pos features with position as key and as value the tuple (token, nPIR)
sen_positions_tokens_score = {}#  Dictionary for sen features with position as key and as value the tuple (token, nPIR)
mlwe_positions_tokens_score = {} # Dictionary for mlwe features with position as key and as value the tuple (token, nPIR)

for le in pos_local_explanations:
    for position, token in le["positions_tokens"].items():
        pos_positions_tokens_score[position] = (token, round(le["nPIR_original_top_class"],4))
        
for le in sen_local_explanations:
    for position, token in le["positions_tokens"].items():
        sen_positions_tokens_score[position] = (token, round(le["nPIR_original_top_class"],4))

for le in mlwe_local_explanations:
    for position, token in le["positions_tokens"].items():
        mlwe_positions_tokens_score[position] = (token, round(le["nPIR_original_top_class"],4))
    

# Utility functions


In [7]:
def label_name(label):
    return ["Negative","Positive"][label]

def get_summary_string(position_token_score):
    positive_color = "background-color:#00FF00"
    negative_color = "background-color:#FF00FF"
    html_string = ""

    for position in sorted(position_token_score.keys(), key=lambda k: int(k)):
        token_score = position_token_score[position]
        if token_score[1] >= 0:
            positive_color = "background-color:rgba(124, 252, 0, {})".format(token_score[1])
            html_string = html_string + '<span style="{}"><b>{}</b></span> '.format(positive_color,token_score[0])
        else:
            negative_color = "background-color:rgba(255, 99, 71, {})".format(token_score[1])
            html_string = html_string + '<span style="{}"><b>{}</b></span> '.format(negative_color,token_score[0])
            
    return html_string

def get_feature_colored_html_string(input_positions_tokens, feature_positions_tokens):
    html_string = ""
    feature_color = "background-color:rgba(0, 255, 255, 1)"
    for position in range(len(input_positions_tokens)):
        if str(position) in feature_positions_tokens:
            html_string = html_string + '<span style="{}"><b>{}</b></span> '.format(feature_color,input_positions_tokens[position])
        else:
            html_string = html_string + '<span><b>{}</b></span> '.format(input_positions_tokens[position])
    return html_string
    

def print_local_explanation(le, input_positions_tokens):
    if len(le["positions_tokens"]) == 0:
        return
    print("LocalExplanationId: {}, FeatureId: {}".format(le["local_explanation_id"], le["feature_id"]))
    print("feature type: ",le["feature_type"])
    print("feature description: ", le["feature_description"])
    print("feature (position,token): " ,le["positions_tokens"])
    print("Perturbed Text:  ",le["perturbed_text"])
    print("perturbed probabilities: ",le["perturbed_probabilities"])
    print("nPIR Original Class: " ,round(le["nPIR_original_top_class"],3))
    display(HTML(get_feature_colored_html_string(input_positions_tokens,le["positions_tokens"])))
    print("\n")
    print("\n")
    return

In [8]:
display(HTML('<h1>Input Text info</h1>'))
display(HTML('Raw Text:   <b>{} </b>'.format(raw_text)))
display(HTML('Clean Text:   <b>{} </b>'.format(cleaned_text)))
display(HTML('Preprocessed Text:   <b>{} </b>'.format(preprocessed_text)))

In [9]:
display(HTML('<h1>Multi-Layer Word Embedding Features Summary</h1>'))
display(HTML('Predicted Class: <b>{} </b>'.format(label_name(input_info["original_label"]))))
display(HTML('Original Probability: <b>{} </b>'.format(input_info["original_probabilities"])))

html_string = get_summary_string(mlwe_positions_tokens_score)
        
display(HTML(html_string))

display(HTML('(Features are highlighted in green if are positively influential for the predicted class, red otherwise. The intensity of the color is proportional to the nPIR score obtained by the word.) '))  


# MLWE Most influential feature

Feature with highest nPIR 

In [10]:
most_influnetial_mlwe_feature =max(mlwe_local_explanations_comb, key= lambda le: le["nPIR_original_top_class"])
print_local_explanation(most_influnetial_mlwe_feature,input_positions_tokens)

LocalExplanationId: 32, FeatureId: 1
feature type:  MLWE
feature description:  Cluster 1
feature (position,token):  {'0': 'this', '1': 'film', '2': 'is', '3': 'very', '4': 'awful', '5': '.', '6': 'i', '11': 'a', '12': 'bad', '13': 'movie', '14': '.'}
Perturbed Text:   have never seen such
perturbed probabilities:  [0.0483090877532959, 0.9516909122467041]
nPIR Original Class:  0.952


<h2>Query Multi-Layer Word Embedding features</h2>

In [11]:
print("Original Probabilities: {}".format(input_info["original_probabilities"]))
print("Original label: {}".format(input_info["original_label"]))
print("\n\n")
min_nPIR_mlwe = -1
max_nPIR_mlwe = 1

for le in sorted(mlwe_local_explanations_comb, key=lambda le: le["nPIR_original_top_class"], reverse=True ):
    if le["nPIR_original_top_class"] >= min_nPIR_mlwe and le["nPIR_original_top_class"] <= max_nPIR_mlwe:
        print_local_explanation(le, input_positions_tokens)
        

Original Probabilities: [0.9983175992965698, 0.001682370901107788]
Original label: 0



LocalExplanationId: 32, FeatureId: 1
feature type:  MLWE
feature description:  Cluster 1
feature (position,token):  {'0': 'this', '1': 'film', '2': 'is', '3': 'very', '4': 'awful', '5': '.', '6': 'i', '11': 'a', '12': 'bad', '13': 'movie', '14': '.'}
Perturbed Text:   have never seen such
perturbed probabilities:  [0.0483090877532959, 0.9516909122467041]
nPIR Original Class:  0.952






LocalExplanationId: 33, FeatureId: 2
feature type:  MLWE
feature description:  Combination of Cluster 0 and Cluster 1
feature (position,token):  {'7': 'have', '8': 'never', '9': 'seen', '10': 'such', '0': 'this', '1': 'film', '2': 'is', '3': 'very', '4': 'awful', '5': '.', '6': 'i', '11': 'a', '12': 'bad', '13': 'movie', '14': '.'}
Perturbed Text:   
perturbed probabilities:  [0.4176056981086731, 0.5823943018913269]
nPIR Original Class:  0.62






LocalExplanationId: 31, FeatureId: 0
feature type:  MLWE
feature description:  Cluster 0
feature (position,token):  {'7': 'have', '8': 'never', '9': 'seen', '10': 'such'}
Perturbed Text:   this film is very awful. i a bad movie.
perturbed probabilities:  [0.9990434646606445, 0.0009565651416778564]
nPIR Original Class:  -0.001


In [12]:
display(HTML('<h1>Parts-of-speech Features Summary</h1>'))
display(HTML('<b>Predicted Class: {} </b>'.format(label_name(input_info["original_label"]))))
display(HTML('<b>Original Probability: {} </b>'.format(input_info["original_probabilities"])))

html_string = get_summary_string(pos_positions_tokens_score)
display(HTML(html_string))

display(HTML('(Features are highlighted in green if are positively influential for the predicted class, red otherwise. The intensity of the color is proportional to the nPIR score obtained by the word.) '))  


# POS Most influential feature

In [13]:
most_influnetial_pos_feature =max(pos_local_explanations_comb, key= lambda le: le["nPIR_original_top_class"])
print_local_explanation(most_influnetial_pos_feature,input_positions_tokens)

LocalExplanationId: 8, FeatureId: 8
feature type:  POS
feature description:  Combination of Adjectives and Verbs
feature (position,token):  {'4': 'awful', '12': 'bad', '2': 'is', '7': 'have', '9': 'seen'}
Perturbed Text:   this film very. i never such a movie.
perturbed probabilities:  [0.0010079145431518555, 0.9989920854568481]
nPIR Original Class:  0.999


<h1>Query Parts-of-speech Features</h1>

In [14]:
print("Original Probabilities: {}".format(input_info["original_probabilities"]))
print("Original label: {}".format(input_info["original_label"]))
print("\n\n")
min_nPIR_pos = -1
max_nPIR_pos = 1

for le in sorted(pos_local_explanations_comb, key=lambda le: le["nPIR_original_top_class"], reverse=True ):
    if le["nPIR_original_top_class"] >= min_nPIR_pos and le["nPIR_original_top_class"] <= max_nPIR_pos:
        print_local_explanation(le, input_positions_tokens)

Original Probabilities: [0.9983175992965698, 0.001682370901107788]
Original label: 0



LocalExplanationId: 8, FeatureId: 8
feature type:  POS
feature description:  Combination of Adjectives and Verbs
feature (position,token):  {'4': 'awful', '12': 'bad', '2': 'is', '7': 'have', '9': 'seen'}
Perturbed Text:   this film very. i never such a movie.
perturbed probabilities:  [0.0010079145431518555, 0.9989920854568481]
nPIR Original Class:  0.999






LocalExplanationId: 0, FeatureId: 0
feature type:  POS
feature description:  Adjectives
feature (position,token):  {'4': 'awful', '12': 'bad'}
Perturbed Text:   this film is very. i have never seen such a movie.
perturbed probabilities:  [0.0010830163955688477, 0.9989169836044312]
nPIR Original Class:  0.999






LocalExplanationId: 11, FeatureId: 11
feature type:  POS
feature description:  Combination of Adjectives and Interjections
feature (position,token):  {'4': 'awful', '12': 'bad'}
Perturbed Text:   this film is very. i have never seen such a movie.
perturbed probabilities:  [0.0010830163955688477, 0.9989169836044312]
nPIR Original Class:  0.999






LocalExplanationId: 12, FeatureId: 12
feature type:  POS
feature description:  Combination of Adjectives and Others (Cardinal Numbers, Foreign Words, Symbols)
feature (position,token):  {'4': 'awful', '12': 'bad'}
Perturbed Text:   this film is very. i have never seen such a movie.
perturbed probabilities:  [0.0010830163955688477, 0.9989169836044312]
nPIR Original Class:  0.999






LocalExplanationId: 10, FeatureId: 10
feature type:  POS
feature description:  Combination of Adjectives and Pronouns, Predeterminers, WH
feature (position,token):  {'4': 'awful', '12': 'bad', '10': 'such'}
Perturbed Text:   this film is very. i have never seen a movie.
perturbed probabilities:  [0.0014573335647583008, 0.9985426664352417]
nPIR Original Class:  0.999


...





LocalExplanationId: 20, FeatureId: 20
feature type:  POS
feature description:  Combination of Verbs and Interjections
feature (position,token):  {'2': 'is', '7': 'have', '9': 'seen'}
Perturbed Text:   this film very awful. i never such a bad movie.
perturbed probabilities:  [0.9990600943565369, 0.0009399056434631348]
nPIR Original Class:  -0.001






LocalExplanationId: 21, FeatureId: 21
feature type:  POS
feature description:  Combination of Verbs and Others (Cardinal Numbers, Foreign Words, Symbols)
feature (position,token):  {'2': 'is', '7': 'have', '9': 'seen'}
Perturbed Text:   this film very awful. i never such a bad movie.
perturbed probabilities:  [0.9990600943565369, 0.0009399056434631348]
nPIR Original Class:  -0.001






LocalExplanationId: 3, FeatureId: 3
feature type:  POS
feature description:  Adverbs, Modals
feature (position,token):  {'3': 'very', '8': 'never'}
Perturbed Text:   this film is awful. i have seen such a bad movie.
perturbed probabilities:  [0.999068021774292, 0.0009319484233856201]
nPIR Original Class:  -0.001






LocalExplanationId: 23, FeatureId: 23
feature type:  POS
feature description:  Combination of Adverbs, Modals and Interjections
feature (position,token):  {'3': 'very', '8': 'never'}
Perturbed Text:   this film is awful. i have seen such a bad movie.
perturbed probabilities:  [0.999068021774292, 0.0009319484233856201]
nPIR Original Class:  -0.001






LocalExplanationId: 24, FeatureId: 24
feature type:  POS
feature description:  Combination of Adverbs, Modals and Others (Cardinal Numbers, Foreign Words, Symbols)
feature (position,token):  {'3': 'very', '8': 'never'}
Perturbed Text:   this film is awful. i have seen such a bad movie.
perturbed probabilities:  [0.999068021774292, 0.0009319484233856201]
nPIR Original Class:  -0.001


In [15]:
display(HTML('<h1>Sentences Features Summary</h1>'))
display(HTML('<b>Predicted Class: {} </b>'.format(label_name(input_info["original_label"]))))
display(HTML('<b>Original Probability: {} </b>'.format(input_info["original_probabilities"])))

html_sen_string = get_summary_string(sen_positions_tokens_score)
        
display(HTML(html_sen_string))

display(HTML('(Features are highlighted in green if are positively influential for the predicted class, red otherwise. The intensity of the color is proportional to the nPIR score obtained by the word.) '))  


# SEN Most influential feature

In [16]:
most_influnetial_sen_feature =max(sen_local_explanations_comb, key= lambda le: le["nPIR_original_top_class"])
print_local_explanation(most_influnetial_sen_feature,input_positions_tokens)

LocalExplanationId: 30, FeatureId: 2
feature type:  SEN
feature description:  Combination of 1° Sentence and 2° Sentence
feature (position,token):  {'0': 'this', '1': 'film', '2': 'is', '3': 'very', '4': 'awful', '5': '.', '6': 'i', '7': 'have', '8': 'never', '9': 'seen', '10': 'such', '11': 'a', '12': 'bad', '13': 'movie', '14': '.'}
Perturbed Text:   
perturbed probabilities:  [0.4176056981086731, 0.5823943018913269]
nPIR Original Class:  0.62


<h1> Query Sentences Features</h1>

In [17]:
print("Original Probabilities: {}".format(input_info["original_probabilities"]))
print("Original label: {}".format(input_info["original_label"]))
print("\n\n")
min_nPIR_sen = -1
max_nPIR_sen = 1

for le in sorted(sen_local_explanations_comb, key=lambda le: le["nPIR_original_top_class"], reverse=True ):
    if le["nPIR_original_top_class"] >= min_nPIR_sen and le["nPIR_original_top_class"] <= max_nPIR_sen:
        print_local_explanation(le, input_positions_tokens)

Original Probabilities: [0.9983175992965698, 0.001682370901107788]
Original label: 0



LocalExplanationId: 30, FeatureId: 2
feature type:  SEN
feature description:  Combination of 1° Sentence and 2° Sentence
feature (position,token):  {'0': 'this', '1': 'film', '2': 'is', '3': 'very', '4': 'awful', '5': '.', '6': 'i', '7': 'have', '8': 'never', '9': 'seen', '10': 'such', '11': 'a', '12': 'bad', '13': 'movie', '14': '.'}
Perturbed Text:   
perturbed probabilities:  [0.4176056981086731, 0.5823943018913269]
nPIR Original Class:  0.62






LocalExplanationId: 28, FeatureId: 0
feature type:  SEN
feature description:  1° Sentence
feature (position,token):  {'0': 'this', '1': 'film', '2': 'is', '3': 'very', '4': 'awful', '5': '.'}
Perturbed Text:   i have never seen such a bad movie.
perturbed probabilities:  [0.9814319610595703, 0.0185680091381073]
nPIR Original Class:  0.033






LocalExplanationId: 29, FeatureId: 1
feature type:  SEN
feature description:  2° Sentence
feature (position,token):  {'6': 'i', '7': 'have', '8': 'never', '9': 'seen', '10': 'such', '11': 'a', '12': 'bad', '13': 'movie', '14': '.'}
Perturbed Text:   this film is very awful.
perturbed probabilities:  [0.9989479780197144, 0.0010520219802856445]
nPIR Original Class:  -0.001
